In [7]:
import os
from PyPDF2 import PdfReader, PdfWriter

In [9]:
# função para verificar se é possível converter string em int
def converte_string_p_numero(num):
    try:
        int(num)
        return True
    except ValueError:
        return False

# função para mapear onde é número em uma lista
def mapeamento_de_lista(lst):
    lst_aux = []
    for elem in lst:
        lst_aux.append(converte_string_p_numero(elem))
    return lst_aux

# Essa função fica responsável por indicar qual o último número de uma lista
## Responsável por encontrar a posição inicial do nome do passageiro
def ultimo_numero_lista(lst_mapeada):
    cont = -1
    for idx, elem in enumerate(lst_mapeada):  # 'enumerate' pega o índice
        if elem:
            cont = idx
    return cont

# Função que mapea o posicionamento da palavra SIM ou NÃO
## Responsável por encontrar a posição final do nome do passageiro
def mapeamento_nao_lst(lst):
    possiveis_valores = ['não', 'nao', 'sim']
    cont = -1
    for elem in lst:
        cont += 1
        if str(elem).lower() in possiveis_valores:
            return cont
            break

# encontra a linha que esta o nome do passageiro
# linha logo acima do cabeçalho "Grupo de Planejamento"
def retorna_linha_captura_passageiro(texto):
    cont = -1
    linhas = texto.split("\n")
    
    for linha in linhas:
        cont += 1
        if "Grupo de Planejamento" in linha:
            cont -= 1
            return cont
            break

In [49]:
#Função
def leitura_rt_petrobras(arquivo):
    #texto, nome_passageiro, linha_passageiro = '', '', 
    #pi_nome_passag, pf_nome_passag = 0, 0
    with open(arquivo, 'rb') as arquivo:
        leitor_pdf = PdfReader(arquivo)
        # Lendo o texto da primeira página
        texto = leitor_pdf.pages[0].extract_text()
        linha_passageiro = texto.split("\n")[retorna_linha_captura_passageiro(texto)].split(" ")
        
        pi_nome_passag = ultimo_numero_lista(mapeamento_de_lista(linha_passageiro)) + 1
        pf_nome_passag = mapeamento_nao_lst(linha_passageiro) - len(linha_passageiro)
        
        nome_passageiro = " ".join(linha_passageiro[pi_nome_passag:pf_nome_passag]).strip()
        num_rt = texto.split("\n")[1].split(" ")[0].strip()
        return num_rt, nome_passageiro

In [51]:
# Caminhos
input_pdf_dir = r"C:\Users\Fernando Guirra\Desktop\SCRIPTS PY\PDF PETROBRAS\RTAgrupada\NOVA"
output_dir = "/Users/Fernando Guirra/Desktop/SCRIPTS PY/PDF PETROBRAS/ArquivosRT/NOVOTESTE/AUTO/"

# Verificar se a pasta de saída existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterar sobre todos os arquivos PDF na pasta
for file_name in os.listdir(input_pdf_dir):
    if file_name.endswith(".pdf"):
        input_pdf_path = os.path.join(input_pdf_dir, file_name)

        # Carregar o arquivo PDF
        reader = PdfReader(input_pdf_path)

        # Separar as páginas do PDF
        for i in range(len(reader.pages)):
            writer = PdfWriter()
            writer.add_page(reader.pages[i])

            # Nome do arquivo de saída (nome do arquivo original + número da página)
            output_pdf_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}pagina{i + 1}.pdf")

            # Salvar o arquivo PDF separado
            with open(output_pdf_path, "wb") as output_pdf:
                writer.write(output_pdf)

with open("Arquivos_nao_processados.txt", "a") as arq_n_processados:  # Modo 'a' para adicionar conteúdo
    arq_n_processados.write("Arquivos que não foram processados \n")
    arq_n_processados.write("\n")

    cont = 0  # Inicializando o contador fora do loop
    for file_rt in os.listdir(output_dir):
        try:
            cont += 1            
            info_rt = leitura_rt_petrobras(output_dir + file_rt)
            nome_arq = "RT " + " ".join(info_rt)
            os.rename(output_dir + file_rt, output_dir + nome_arq + '.pdf')
        
        except Exception as e:
            arq_n_processados.write(f"RT {file_rt} \t --- \t Falha: {str(e)}\n")
        
        finally:
            arq_n_processados.close()
            print("PDFs processados, separados e renomeados com sucesso!")